# Introduction

https://towardsdatascience.com/how-to-structure-your-data-science-notebook-to-be-easy-to-follow-2d3c2777e6e0

- describe the project in terms of business goals
- give context to the work, where it originated, and what you want to achieve.
- briefly talk about any prior knwledge (For example, if the sales data is only from one specific store, that should be mentioned. If at a certain period of time the company had problems with some of the products, like distribution issues, directly affecting sales, this should be stated too. Basically, we have to describe anything that helps understand the context of the data sources and important details.)

This project leverages the OKCupid Profiles Dataset (sourced from Kaggle) to explore, analyze, and clean data for building a content-filtering system. The primary goal is to recommend potential matches based on user preferences and attributes. The dataset does not include user images, but contains various user attributes, including demographics, interests, and lifestyle choices as well as 10 "essays" which are just answers to dating prompts such as "Dating me looks like..." and "Together we can..."


# Data Wrangling

This section focuses on importing the OKCupid dataset from Kaggle, looking at the comprehensive overview and descriptive statistics. I will then look at the amount of data missing.

In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from datetime import datetime, timedelta
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import kagglehub
# from kagglehub import KaggleDatasetAdapter
# Download latest version
path = kagglehub.dataset_download("andrewmvd/okcupid-profiles")

print("Path to dataset files:", path)

100%|██████████| 50.6M/50.6M [00:00<00:00, 68.2MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/andrewmvd/okcupid-profiles/versions/1


In [ ]:
df = pd.read_csv('/root/.cache/kagglehub/datasets/andrewmvd/okcupid-profiles/versions/1/okcupid_profiles.csv', sep=",",header=0)
df.sample(5)

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
3245,25,single,m,straight,fit,mostly anything,socially,never,graduated from college/university,white,...,"bay area native, recent ma resident, college g...","working, furnishing my apartment, settling in",making cereal,my sweet dance moves,books: opened ground -- seamus heaney; a farew...,1. my glasses 2. my contacts 3. sleep 4. tea -...,"running and how i'd like to do it more, philos...",perhaps: sleeping reading with friends (maybe ...,i've been to a small-town rodeo.,you're interested you're smart you wouldn't be...
42085,27,single,m,gay,average,other,often,never,NaN,asian,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47871,22,single,m,straight,athletic,vegetarian,NaN,never,working on college/university,asian,...,i'm just a simple honest guy. i consider mysel...,trying to finish up college. major in business...,"cooking, martial arts, taking photos, video ga...",i'm asian.... sad ain't it...? that's all peop...,lets save this for when we meet in person. con...,"my camera my computer, my friends, my martial ...","life, philosophy, secrets to life, work, photo...",listening to music in my room or failing at te...,i'm teaching myself how to dance in my room wi...,you're into asian guys (particularly if you're...
28426,24,single,m,straight,fit,mostly anything,socially,never,working on college/university,asian,...,i lived in the bay area most of my life. after...,getting my degree in finance. i have a year left.,i'd like to think i'm a pretty well rounded pe...,i have a real laid back attitude,"books: the help, one day, myth of sisyphus, th...",friends/family food/water internet books,where i want to travel to next.,"hanging out with friends but lately, i've been...",NaN,you feel like it
51409,58,single,m,straight,fit,NaN,rarely,never,graduated from college/university,white,...,"i could say i am funny, especially in reaction...",at the moment i am converting a structure on m...,"let see, coffee roasting and tofu making. mach...","my openess, my hi level energy, my sense of hu...","books, so many, but i love russian classics. h...","love, music, coffee, books, the tropics, got a...","the kids, the enviroment, improve myself-all a...",either spending time with friend at a dance cl...,"ask me, but promise to forgive me.",you can use a keyboard.


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          59946 non-null  int64  
 1   status       59946 non-null  object 
 2   sex          59946 non-null  object 
 3   orientation  59946 non-null  object 
 4   body_type    54650 non-null  object 
 5   diet         35551 non-null  object 
 6   drinks       56961 non-null  object 
 7   drugs        45866 non-null  object 
 8   education    53318 non-null  object 
 9   ethnicity    54266 non-null  object 
 10  height       59943 non-null  float64
 11  income       59946 non-null  int64  
 12  job          51748 non-null  object 
 13  last_online  59946 non-null  object 
 14  location     59946 non-null  object 
 15  offspring    24385 non-null  object 
 16  pets         40025 non-null  object 
 17  religion     39720 non-null  object 
 18  sign         48890 non-null  object 
 19  smok

In [ ]:
df.describe()

,age,height,income
count,59946.000000,59943.000000,59946.000000
mean,32.340290,68.295281,20033.222534
std,9.452779,3.994803,97346.192104
min,18.000000,1.000000,-1.000000
25%,26.000000,66.000000,-1.000000
50%,30.000000,68.000000,-1.000000
75%,37.000000,71.000000,-1.000000
max,110.000000,95.000000,1000000.000000


In [ ]:
# Calculate the percentage of missing values for each column
missing_summary = df.isnull().agg(['sum', 'mean']).T

# Rename columns for clarity
missing_summary.columns = ['# Missing Values', '% Missing']

# Format percentage as a string with two decimal places
missing_summary['% Missing'] = (missing_summary['% Missing'] * 100).apply(lambda x: f"{x:.2f}%")

# Display the result
print(missing_summary)

             # Missing Values % Missing
age                       0.0     0.00%
status                    0.0     0.00%
sex                       0.0     0.00%
orientation               0.0     0.00%
body_type              5296.0     8.83%
diet                  24395.0    40.69%
drinks                 2985.0     4.98%
drugs                 14080.0    23.49%
education              6628.0    11.06%
ethnicity              5680.0     9.48%
height                    3.0     0.01%
income                    0.0     0.00%
job                    8198.0    13.68%
last_online               0.0     0.00%
location                  0.0     0.00%
offspring             35561.0    59.32%
pets                  19921.0    33.23%
religion              20226.0    33.74%
sign                  11056.0    18.44%
smokes                 5512.0     9.19%
speaks                   50.0     0.08%
essay0                 5488.0     9.15%
essay1                 7572.0    12.63%
essay2                 9638.0    16.08%


The sample showed me rows with a lot of missing information. I want to take a deeper look to see if it is worth dropping rows with more than half of the features missing as there is no way to fill in the information lost.

In [ ]:
# Count the number of NaN values per row
nan_counts_per_row = df.isnull().sum(axis=1)

# Define a threshold (e.g., more than half the columns)
threshold = df.shape[1] // 2

# Count the rows with NaN values exceeding the threshold
rows_above_threshold = (nan_counts_per_row > threshold).sum()

# Display the count
print(f"Number of rows with NaNs exceeding the threshold of {threshold}: {rows_above_threshold}")

percent_missing = (rows_above_threshold / len(df)) * 100
print(f"Percentage of rows with NaNs exceeding the threshold of {threshold}: {percent_missing:.2f}%")

Number of rows with NaNs exceeding the threshold of 15: 931
Percentage of rows with NaNs exceeding the threshold of 15: 1.55%


1.45% of the data contains rows with more than 15 features missing. I will remove them as dropping them will not impact the dataset.

In [ ]:
df = df[nan_counts_per_row <= threshold]

In [ ]:
df.sample(10)

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
37777,24,single,m,straight,thin,mostly anything,socially,sometimes,graduated from college/university,white,...,here is a list of some interesting facts about...,"teacher, currently in a preschool.","witty quips and puns. also, asking questions.",i've heard people say they notice my eyes. i l...,"i love books with extreme passion. recently, s...",-being near a community of people that i love ...,what i could do better...being hard on yoursel...,"hanging out with friends at someone's house, e...","i raise some of my animals for meat, and it's ...",you want to hang out.
27002,23,single,f,gay,thin,strictly vegetarian,socially,never,working on college/university,white,...,some say that i act like a teenage boy. i'm no...,i'm finishing up my teaching certificate and a...,making indian food. getting people to laugh wh...,i've been told i have an adorable nose.,"i'm a big fan of battlestar galactica, but my ...",NaN,how to re-tie the knot on my mocassins.,"studying for a test, then heading out to a fri...",i hardly ever wear underwear. i think it's tot...,you're feeling lucky.
58779,36,single,m,straight,average,mostly anything,rarely,never,working on college/university,hispanic / latin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1262,29,single,m,straight,athletic,mostly anything,desperately,never,graduated from college/university,"asian, middle eastern, black, native american,...",...,grew up in east texas and speaks southern yall...,i speak to think and generally say what's on m...,"watching tv while i shower, knocking over tabl...","i often told i look like one of 2 celebrities,...",the bk hipsters squad on here needs to pump th...,air conditioning cold showers headphones bacon...,planning my next trip. food.,watching a billion movies and getting drunk in...,"on one of my dates off here, i got my jacket s...",honestly though - if you're just a normal pers...
2209,27,single,m,straight,average,anything,socially,never,graduated from college/university,asian,...,born and grew up in hong kong. live in san fra...,working as an application engineer in super mi...,"piano, ice-skating, and cooking.",lots of people say i look like a celebraty in ...,"history books, classical & pop music......","eating, drinking, and breathing. updating myse...",what other people are thinking. how rich peopl...,hanging out social with friends and coworkers.,chubby girls are acceptable for me.,NaN
10052,31,single,m,straight,fit,NaN,not at all,never,graduated from masters program,"asian, other",...,enjoying life with family and friends! i enjoy...,i am passionate about public health and commun...,NaN,NaN,food: anything with chicken...and anything swe...,NaN,NaN,NaN,that i'm here on okcupid of course!,"to hang out, have a great time, and get to kno..."
49685,35,available,f,bisexual,thin,NaN,socially,never,working on masters program,white,...,"mae west's mantra, you only live once, but if ...",i teach high school and love it! before i was...,"finding the connections, helping people become...",my eyes? my wavy/curly hair? my butt? i can't ...,"books: eat pray love, king dork, caucasia, mid...",1. chapstick (it's kind of a security blanket ...,changing up power dynamics in our society... a...,exhausted and nearly falling asleep wherever i...,uh-uh,you are passionate and are trying to do someth...
49007,26,single,f,straight,athletic,NaN,often,never,NaN,asian,...,just joined...,trying to have as much fun as possible. :p,NaN,my eyes,NaN,NaN,NaN,out partying,NaN,NaN
29448,30,single,f,straight,average,NaN,often,sometimes,graduated from college/university,NaN,...,i'm an east coast transplant who fell pretty h...,i love the sf food scene and am lucky to have ...,being rational and telling people how much i l...,my curls and my laugh.,"book : atlas shrugged movie : limitless, once,...",a sense of humor close friends water coff

## Numerical Columns

The numerical columns include age, height, and income.

### Age Column

In [ ]:
print("Age Min", df['age'].min())
print("Age Max", df['age'].max())

Age Min 18
Age Max 109


**Handling Outliers in Age Column**
- The minimum age is acceptable and doesn't require changes.
- The maximum age needs to be addressed as it exceeds realistic limits.
- Further steps:
  1. Investigate rows with ages over 100.
  2. Determine whether to remove or impute those values.

In [ ]:
df['age'].value_counts().sort_index()

,count
age,
18,298
19,582
20,925
21,1246
22,1895
23,2546
24,3189
25,3497
26,3663


Looked closer at ages 109, and 110 and decided to drop them due to a lot of missing data that cannot be filled in.

In [ ]:
df[(df['age'] == 109) | (df['age'] == 110)]

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
25324,109,available,m,straight,athletic,mostly other,NaN,never,working on masters program,NaN,...,NaN,NaN,NaN,nothing,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# filtering the data to no longer include ages 109 and 110

df = df[(df['age'] != 109) & (df['age'] != 110)]

# verifying the changes
df['age'].value_counts().sort_index()

,count
age,
18,298
19,582
20,925
21,1246
22,1895
23,2546
24,3189
25,3497
26,3663


The `age` column does not have any missing values, so we will move on to the `height` column.

### Height Column

In [ ]:
print("Min Height:",df['height'].min())
print("Max Height:",df['height'].max())

Min Height: 1.0
Max Height: 95.0


In [ ]:
df['height'].value_counts().sort_index()

,count
height,
1.0,1
3.0,1
4.0,1
6.0,1
8.0,1
9.0,1
26.0,1
36.0,9
37.0,2


There is no information on the unit for height. In this case, it makes sense to assume the height is in inches. I will filter out anomalies and the lowest recorded height will be 4'9" (59 inches) to 6'6" (80 inches).

In [ ]:
df = df[(df['height'] >= 59) & (df['height'] <= 80)]

In [ ]:
print("Min Height:",df['height'].min())
print("Max Height:",df['height'].max())

Min Height: 59.0
Max Height: 80.0


In [ ]:
df.sample(2)

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
7436,21,single,f,straight,average,vegetarian,socially,NaN,working on two-year college,other,...,heres a few things that keep me entertained......,fashion student. works in retail. &blogger; ww...,making something look visually pleasing; im pr...,my piercings & style.,"movies: moulin rouge, eternal sunshine of the ...",beauty/vision. truth. freedom. love. humor. ca...,"change!; trends; the past, present & future (a...",there is no 'typical friday night',i think i revealed enough about myself already.,you think you can hang.
7743,32,single,m,gay,athletic,mostly anything,socially,NaN,working on two-year college,white,...,roseanne barr for president 2012! (green party...,i am a freshly-licensed emt-b. next up more fi...,"cuddling, making out, sounding authoritative, ...","my ass. it's really big for a white boy, and i...","bcher: guns, germs & steel; the third chimpanz...","integrity, moral character, work ethic, pride,...",cardiac tamponade and beck's triad. tension he...,"either working medical at a show, at the bar w...",i watch straight porn sometimes. there! i said...,you like this video of me selfing myself ;) (i...


The height column has 3 missing values, therefore I will drop it as there is no way to fill in the height with accuracy.

In [ ]:
df.dropna(subset=['height'], inplace=True)

### Income

In [ ]:
print("Min Income:",df['income'].min())
print("Max Income:",df['income'].max())

Min Income: -1
Max Income: 1000000


In [ ]:
df['income'].value_counts().sort_index()

,count
income,
-1,47399
20000,2929
30000,1043
40000,1000
50000,970
60000,736
70000,702
80000,1108
100000,1613


A large portion of the income values are "-1". Dropping it will incur a large loss of data, and keeping it numerical might create noise. To mitigate this, I will create categorical bins to assign -1s as "Unknown" to keep the -1s.

In [ ]:
# income categorization
def categorize_income(income):
    if income == -1:
        return "Unknown"
    elif income <= 50000:
        return "Low Income"
    elif income <= 100000:
        return "Medium Income"
    elif income <= 250000:
        return "High Income"
    else:
        return "Very High Income"

df['income_category'] = df['income'].apply(categorize_income)

print(df['income_category'].value_counts())


income_category
Unknown             47399
Low Income           5942
Medium Income        4159
High Income           771
Very High Income      551
Name: count, dtype: int64


## Categorical Columns

Categorical columns include demographics, interests, and lifestyle choices as well as 10 "essays" which are just answers to dating prompts.

### Status

The status column has no missing values, therefore cleaning it will be simple.

In [ ]:
# checking that the values in status are standardized
df['status'].value_counts()

,count
status,
single,54642
seeing someone,2032
available,1842
married,298
unknown,8


`single` has the most values followed by `seeing someone`. `available` and single can be merged together since there is overlap.

In [ ]:
df['status'] = df['status'].replace({'single': 'single', 'available': 'single'})

print(df['status'].value_counts())

status
single            56484
seeing someone     2032
married             298
unknown               8
Name: count, dtype: int64


### Sex

In [ ]:
# checking for missing values as well as making sure everything is standardized.
df['sex'].value_counts()

,count
sex,
m,35091
f,23731


### Orientation

In [ ]:
df['orientation'].value_counts()

,count
orientation,
straight,50613
gay,5495
bisexual,2714


### Body Type

In [ ]:
df['body_type'].value_counts()

,count
body_type,
average,14436
fit,12526
athletic,11608
thin,4658
curvy,3883
a little extra,2606
skinny,1733
full figured,1005
overweight,440


In [ ]:
df['body_type'] = df['body_type'].replace({'fit': 'fit/athletic', 'athletic': 'fit/athletic', 'jacked': 'fit/athletic', 'skinny': 'thin', 'curvy': 'curvy/full figured', 'full figured': 'curvy/full figured', 'a little extra': 'overweight'})

print(df['body_type'].value_counts())

body_type
fit/athletic          24537
average               14436
thin                   6391
curvy/full figured     4888
overweight             3046
used up                 338
rather not say          191
Name: count, dtype: int64


### Diet

Diet has 24395 missing values which affects 40.9% of the data. Let's take a look.

In [ ]:
df['diet'].value_counts()

,count
diet,
mostly anything,16493
anything,6128
strictly anything,5088
mostly vegetarian,3434
mostly other,1000
strictly vegetarian,869
vegetarian,662
strictly other,444
mostly vegan,337


In [ ]:
df['diet'].sample(10)

,diet
30184,mostly anything
3798,mostly vegetarian
37730,mostly anything
24185,strictly vegetarian
42000,anything
30657,NaN
1205,NaN
25770,NaN
41411,mostly vegetarian
19349,mostly vegetarian


In [ ]:
diet_mapping = {
    'anything': 'anything',
    'mostly anything': 'anything',
    'strictly anything': 'anything',
    'vegetarian': 'vegetarian',
    'mostly vegetarian': 'vegetarian',
    'strictly vegetarian': 'vegetarian',
    'vegan': 'vegan',
    'mostly vegan': 'vegan',
    'strictly vegan': 'vegan',
    'other': 'other',
    'mostly other': 'other',
    'strictly other': 'other',
    'kosher': 'kosher',
    'mostly kosher': 'kosher',
    'strictly kosher': 'kosher',
    'halal': 'halal',
    'mostly halal': 'halal',
    'strictly halal': 'halal'
}

df['diet'] = df['diet'].map(diet_mapping).fillna('unknown')

In [ ]:
df['diet'].value_counts()

,count
diet,
anything,27709
unknown,23486
vegetarian,4965
other,1772
vegan,701
kosher,113
halal,76


In [ ]:
df['diet'].sample(5)

,diet
41524,unknown
8072,anything
30118,anything
42580,unknown
48696,unknown


### Drinks/Drugs/Smoke

The `drinks` column has 2,985 missing values which represents 4.98% of the data.
`drugs` has 14,080 missing values which represents 23.49% of data.
`smokes` has 5,512 missing values which represents 9.19% of data.

In [ ]:
df['drinks'].value_counts()

,count
drinks,
socially,41376
rarely,5901
often,5104
not at all,3226
very often,457
desperately,312


In [ ]:
drinks_mapping = {
    'socially': 'socially',
    'rarely': 'rarely/not at all',
    'not at all': 'rarely/not at all',
    'often': 'often',
    'very often': 'often',
    'desperately': 'often'
}

df['drinks'] = df['drinks'].map(drinks_mapping).fillna('unknown')

print(df['drinks'].value_counts())

drinks
socially             41376
rarely/not at all     9127
often                 5873
unknown               2446
Name: count, dtype: int64


In [ ]:
# drugs
df['drugs'].value_counts()

,count
drugs,
never,36972
sometimes,7665
often,392


In [ ]:
df['drugs'] = df['drugs'].fillna('unknown')
print(df['drugs'].isnull().sum())

0


In [ ]:
# smokes
df['smokes'].value_counts()

,count
smokes,
no,43484
sometimes,3739
when drinking,3008
yes,2196
trying to quit,1471


In [ ]:
smokes_mapping = {
    'no': 'no',
    'sometimes': 'occasionally',
    'when drinking': 'occasionally',
    'yes': 'regularly',
    'trying to quit': 'trying to quit'
}

df['smokes'] = df['smokes'].map(smokes_mapping).fillna('unknown')

print(df['smokes'].value_counts())

smokes
no                43484
occasionally       6747
unknown            4924
regularly          2196
trying to quit     1471
Name: count, dtype: int64


### Education

6,628 values missing

In [ ]:
df['education'].value_counts()

,count
education,
graduated from college/university,23816
graduated from masters program,8901
working on college/university,5672
working on masters program,1674
graduated from two-year college,1525
graduated from high school,1413
graduated from ph.d program,1264
graduated from law school,1111
working on two-year college,1063


In [ ]:
# Define mapping for education levels
education_mapping = {
    'graduated from college/university': 'college/university_graduated',
    'working on college/university': 'college/university_studying',
    'dropped out of college/university': 'college/university_dropped out',
    'graduated from masters program': 'masters_graduated',
    'working on masters program': 'masters_studying',
    'dropped out of masters program': 'masters_dropped out',
    'graduated from ph.d program': 'ph.d_graduated',
    'working on ph.d program': 'ph.d_studying',
    'dropped out of ph.d program': 'ph.d_dropped out',
    'graduated from law school': 'law school_graduated',
    'working on law school': 'law school_studying',
    'dropped out of law school': 'law school_dropped out',
    'graduated from med school': 'med school_graduated',
    'working on med school': 'med school_studying',
    'dropped out of med school': 'med school_dropped out',
    'graduated from two-year college': 'two-year college_graduated',
    'working on two-year college': 'two-year college_studying',
    'dropped out of two-year college': 'two-year college_dropped out',
    'graduated from high school': 'high school_graduated',
    'working on high school': 'high school_studying',
    'dropped out of high school': 'high school_dropped out',
    'space camp': 'other',
    'working on space camp': 'other',
    'dropped out of space camp': 'other',
    'graduated from space camp': 'other'
}

# Apply the mapping
df['education'] = df['education'].map(education_mapping).fillna('unknown')

# Verify the result
print(df['education'].value_counts())


education
college/university_graduated      23816
masters_graduated                  8901
unknown                            7158
college/university_studying        5672
masters_studying                   1674
other                              1664
two-year college_graduated         1525
high school_graduated              1413
ph.d_graduated                     1264
law school_graduated               1111
two-year college_studying          1063
college/university_dropped out      993
ph.d_studying                       975
med school_graduated                443
law school_studying                 268
med school_studying                 211
two-year college_dropped out        191
masters_dropped out                 140
ph.d_dropped out                    126
high school_dropped out              98
high school_studying                 87
law school_dropped out               17
med school_dropped out               12
Name: count, dtype: int64


In [ ]:
print(df['education'].isnull().sum())

0


### Ethnicity

In [ ]:
# checking the values
print(df['ethnicity'].value_counts())

ethnicity
white                                              32495
asian                                               6019
hispanic / latin                                    2763
black                                               1974
other                                               1669
                                                   ...  
black, native american, indian, white                  1
black, native american, pacific islander, other        1
asian, middle eastern, black, pacific islander         1
middle eastern, black, pacific islander, white         1
asian, black, indian                                   1
Name: count, Length: 217, dtype: int64


Since there are over 217 different ethnicity combinations in this dataset, it's best to standardize the ethnicity column to reduce ambiguity.

In [ ]:
def standardize_ethnicity(value):
    if pd.isna(value):  # if missing,
        return 'unknown'
    # Split by comma, strip whitespace, convert to lowercase, and remove duplicates
    ethnicities = sorted(set([eth.strip().lower() for eth in value.split(',')]))
    # Combine back into a standardized string
    return ', '.join(ethnicities)

# Apply the cleaning function to the 'ethnicity' column
df['ethnicity'] = df['ethnicity'].apply(standardize_ethnicity)

# Group rare combinations into a 'mixed' category (optional)
common_ethnicities = ['white', 'asian', 'black', 'hispanic / latin', 'native american', 'pacific islander', 'middle eastern']
df['ethnicity'] = df['ethnicity'].apply(
    lambda x: x if x in common_ethnicities else ('mixed' if ',' in x else x)
)

# Verify the cleaned and standardized column
print(df['ethnicity'].value_counts())


ethnicity
white               32495
mixed                6778
asian                6019
unknown              5262
hispanic / latin     2763
black                1974
other                1669
indian               1062
pacific islander      413
middle eastern        324
native american        63
Name: count, dtype: int64


### Job

In [ ]:
print(df['job'].value_counts())

job
other                                7547
student                              4851
science / tech / engineering         4825
computer / hardware / software       4682
artistic / musical / writer          4410
sales / marketing / biz dev          4373
medicine / health                    3659
education / academia                 3497
executive / management               2357
banking / financial / real estate    2240
entertainment / media                2234
law / legal services                 1369
hospitality / travel                 1352
construction / craftsmanship         1016
clerical / administrative             801
political / government                697
rather not say                        431
transportation                        363
unemployed                            270
retired                               246
military                              201
Name: count, dtype: int64


Although the job column is useful in its current form. The best thing to do is fill in the NaNs as 'rather not say'.

In [ ]:
df['job'] = df['job'].fillna('rather not say')
print(df['job'].value_counts())

job
rather not say                       7832
other                                7547
student                              4851
science / tech / engineering         4825
computer / hardware / software       4682
artistic / musical / writer          4410
sales / marketing / biz dev          4373
medicine / health                    3659
education / academia                 3497
executive / management               2357
banking / financial / real estate    2240
entertainment / media                2234
law / legal services                 1369
hospitality / travel                 1352
construction / craftsmanship         1016
clerical / administrative             801
political / government                697
transportation                        363
unemployed                            270
retired                               246
military                              201
Name: count, dtype: int64


### Last Online
Last Online doesn't have any missing data, but I still want to take a look at the column to look at the structure.

In [ ]:
print(df['last_online'].sample(10))

56527    2012-06-30-17-24
30078    2012-06-27-08-47
13237    2012-06-29-10-40
20669    2012-06-27-11-54
9260     2012-06-30-00-08
22724    2012-06-29-21-58
57030    2012-04-21-10-05
59633    2012-07-01-01-58
7214     2012-06-16-22-38
34530    2012-06-16-23-40
Name: last_online, dtype: object


Now that I've looked into it, I want to convert this to 'days ago' in which the user was last online. Ex: 4 days ago.

In [ ]:
latest_date = pd.to_datetime(df['last_online'].max(), format='%Y-%m-%d-%H-%M') # Specify format for latest_date
df['last_online'] = pd.to_datetime(df['last_online'], format='%Y-%m-%d-%H-%M', errors='coerce') # Add errors='coerce' to handle invalid dates

# Categorize users based on activity recency
df['activity_period'] = pd.cut(
    (latest_date - df['last_online']).dt.days,
    bins=[-1, 365, 5*365, float('inf')],
    labels=['Active in last year', 'Active 1-5 years ago', 'Inactive']
)

# View distribution
print(df['activity_period'].value_counts())


activity_period
Active in last year     58737
Active 1-5 years ago       85
Inactive                    0
Name: count, dtype: int64


In [ ]:
df.sample(5)

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,income_category,activity_period
6638,27,single,m,straight,fit/athletic,vegetarian,socially,never,other,white,...,making people feel comfortable. being very low...,my dimples or big brown eyes.,NaN,i don't think anyone takes this one seriously....,NaN,glad to be in cali.,i groom myself head to toe.,...you feel like it? you need a friend? a love...,Unknown,Active in last year
26459,27,seeing someone,f,straight,NaN,unknown,socially,never,law school_studying,middle eastern,...,-making people feel comfortable opening up to ...,"not really sure. i've got kind of a big nose, ...",books: i like all sorts of stuff that's not co...,1. water 2. fruit 3. cuddles 4. good conversat...,power dynamics - in cultures and societies and...,friday's my day off so i'm typically relaxing ...,i just got a new car and opted for a manual an...,if you feel like chatting. if you're not expec...,Unknown,Active in last year
19083,40,single,m,straight,fit/athletic,anything,socially,never,college/university_graduated,white,...,being messy and clean at the same time. don't ...,"aside from my appearance? well, probably my se...",some of my favorite movies are: the big lebows...,"highlighters, shoehorn, wikipedia, burritos, p...","my life, what's most important in life, what t...","either having dinner out, and maybe seeing a m...","i don't really like squash, the food or the game!","you're down to earth, a little quirky, have a ...",Medium Income,Active in last year
37580,31,single,f,bisexual,curvy/full figured,unknown,socially,never,college/university_graduated,hispanic / latin,...,eating a new dish and figuring out what goes i...,NaN,"books: down these mean streets, the alchemist,...",NaN,"my career dreams/goals, where i want to travel...","relaxing at home after a crazy week, but i'd l...",NaN,NaN,Unknown,Active in last year
53899,53,single,f,straight,NaN,anything,socially,never,masters_graduated,unknown,...,seeing the big picture being real laughing and...,"my hair, my smile",kitchen table wisdom - rachel naomi remen; wal...,my kids love and connection with family and fr...,NaN,"relaxing, hanging out with my son, having dinn...",NaN,you think we might click and have something to...,Unknown,Active in last year


### Location

In [ ]:
df['location'].value_counts()

,count
location,
"san francisco, california",30514
"oakland, california",7107
"berkeley, california",4150
"san mateo, california",1309
"palo alto, california",1052
...,...
"jackson, mississippi",1
"ozone park, new york",1
"lake orion, michigan",1


We're not able to see all the locations, but we're going to go ahead and standardized.

In [ ]:
df['location'] = df['location'].str.lower().str.strip()

### Offspring

In [ ]:
df['offspring'].value_counts()

,count
offspring,
doesn't have kids,7509
"doesn't have kids, but might want them",3859
"doesn't have kids, but wants them",3554
doesn't want kids,2909
has kids,1874
has a kid,1869
"doesn't have kids, and doesn't want any",1128
"has kids, but doesn't want more",440
"has a kid, but doesn't want more",274


In [ ]:
offspring_mapping = {
    "doesn't have kids": "no kids, no preference",
    "doesn't have kids, but might want them": "no Kids, might want",
    "doesn't have kids, but wants them": "no kids, wants",
    "doesn't want kids": "no kids, doesn't want",
    "has kids": "has kids, no preference",
    "has a kid": "has kids, no preference",
    "doesn't have kids, and doesn't want any": "no kids, doesn't want",
    "has kids, but doesn't want more": "has kids, doesn't want more",
    "has a kid, but doesn't want more": "has kids, doesn't want more",
    "has a kid, and might want more": "has kids, might want more",
    "wants kids": "wants kids",
    "might want kids": "might want kids",
    "has kids, and might want more": "has kids, might want more",
    "has a kid, and wants more": "has kids, wants more",
    "has kids, and wants more": "has kids, wants more"
}

df['offspring'] = df['offspring'].map(offspring_mapping).fillna('unknown')

In [ ]:
df['offspring'].sample(15)

,offspring
7445,"no kids, wants"
38623,"no kids, doesn't want"
53661,unknown
45103,"no kids, no preference"
52163,"no kids, wants"
57082,unknown
10969,"no kids, no preference"
18591,unknown
33275,"no kids, no preference"
51620,"no kids, no preference"


### Pets

In [ ]:
df['pets'].value_counts()

,count
pets,
likes dogs and likes cats,14753
likes dogs,7191
likes dogs and has cats,4293
has dogs,4096
has dogs and likes cats,2324
likes dogs and dislikes cats,2022
has dogs and has cats,1464
has cats,1395
likes cats,1057


In [ ]:
pets_mapping = {
    "likes dogs and likes cats": "likes pets",
    "likes dogs": "likes dogs",
    "likes cats": "likes cats",
    "likes dogs and has cats": "has pets",
    "has dogs": "has pets",
    "has dogs and likes cats": "has pets",
    "has dogs and has cats": "has pets",
    "has cats": "has pets",
    "has dogs and dislikes cats": "has pets",
    "likes dogs and dislikes cats": "likes dogs",
    "dislikes dogs and likes cats": "likes cats",
    "dislikes dogs and dislikes cats": "dislikes pets",
    "dislikes cats": "dislikes pets",
    "dislikes dogs": "dislikes pets",
    "dislikes dogs and has cats": "has pets"
}

# Apply mapping directly to the 'pets' column
df['pets'] = df['pets'].map(pets_mapping).fillna("unknown")

# Verify changes
print(df['pets'].value_counts())

pets
unknown          19006
likes pets       14753
has pets         14201
likes dogs        9213
likes cats        1292
dislikes pets      357
Name: count, dtype: int64


### Religion

In [ ]:
df['religion'].value_counts()

,count
religion,
agnosticism,2701
other,2671
agnosticism but not too serious about it,2631
agnosticism and laughing about it,2488
catholicism but not too serious about it,2304
atheism,2166
other and laughing about it,2108
atheism and laughing about it,2067
christianity but not too serious about it,1945


In [ ]:
# Define a dictionary-based mapping for standardizing religion
religion_mapping = {
    "agnosticism": "agnosticism",
    "agnosticism but not too serious about it": "agnosticism",
    "agnosticism and laughing about it": "agnosticism",
    "agnosticism and somewhat serious about it": "agnosticism",
    "agnosticism and very serious about it": "agnosticism",
    "atheism": "atheism",
    "atheism but not too serious about it": "atheism",
    "atheism and laughing about it": "atheism",
    "atheism and somewhat serious about it": "atheism",
    "atheism and very serious about it": "atheism",
    "christianity": "christianity",
    "christianity but not too serious about it": "christianity",
    "christianity and laughing about it": "christianity",
    "christianity and somewhat serious about it": "christianity",
    "christianity and very serious about it": "christianity",
    "catholicism": "catholicism",
    "catholicism but not too serious about it": "catholicism",
    "catholicism and laughing about it": "catholicism",
    "catholicism and somewhat serious about it": "catholicism",
    "catholicism and very serious about it": "catholicism",
    "judaism": "judaism",
    "judaism but not too serious about it": "judaism",
    "judaism and laughing about it": "judaism",
    "judaism and somewhat serious about it": "judaism",
    "judaism and very serious about it": "judaism",
    "buddhism": "buddhism",
    "buddhism but not too serious about it": "buddhism",
    "buddhism and laughing about it": "buddhism",
    "buddhism and somewhat serious about it": "buddhism",
    "buddhism and very serious about it": "buddhism",
    "islam": "islam",
    "islam but not too serious about it": "islam",
    "islam and laughing about it": "islam",
    "islam and somewhat serious about it": "islam",
    "islam and very serious about it": "islam",
    "hinduism": "hinduism",
    "hinduism but not too serious about it": "hinduism",
    "hinduism and laughing about it": "hinduism",
    "hinduism and somewhat serious about it": "hinduism",
    "hinduism and very serious about it": "hinduism",
}

# Apply mapping directly to 'religion' column
df['religion'] = df['religion'].str.lower().map(religion_mapping).fillna("other")

# Verify results
print(df['religion'].value_counts())


religion
other           27014
agnosticism      8772
atheism          6964
christianity     5748
catholicism      4726
judaism          3080
buddhism         1937
hinduism          446
islam             135
Name: count, dtype: int64


### Astrological Sign

In [ ]:
df['sign'].value_counts()

,count
sign,
gemini and it&rsquo;s fun to think about,1771
scorpio and it&rsquo;s fun to think about,1760
leo and it&rsquo;s fun to think about,1680
libra and it&rsquo;s fun to think about,1639
taurus and it&rsquo;s fun to think about,1634
cancer and it&rsquo;s fun to think about,1590
pisces and it&rsquo;s fun to think about,1583
sagittarius and it&rsquo;s fun to think about,1575
virgo and it&rsquo;s fun to think about,1568


In [ ]:
sign_mapping = {
    "leo": "leo",
    "leo but not too serious about it": "leo",
    "leo and laughing about it": "leo",
    "leo and somewhat serious about it": "leo",
    "leo and very serious about it": "leo",
    "libra": "libra",
    "libra but not too serious about it": "libra",
    "libra and laughing about it": "libra",
    "libra and somewhat serious about it": "libra",
    "libra and very serious about it": "libra",
    "cancer": "cancer",
    "cancer but not too serious about it": "cancer",
    "cancer and laughing about it": "cancer",
    "cancer and somewhat serious about it": "cancer",
    "cancer and very serious about it": "cancer",
    "virgo": "virgo",
    "virgo but not too serious about it": "virgo",
    "virgo and laughing about it": "virgo",
    "virgo and somewhat serious about it": "virgo",
    "virgo and very serious about it": "virgo",
    "scorpio": "scorpio",
    "scorpio but not too serious about it": "scorpio",
    "scorpio and laughing about it": "scorpio",
    "scorpio and somewhat serious about it": "scorpio",
    "scorpio and very serious about it": "scorpio",
    "gemini": "gemini",
    "gemini but not too serious about it": "gemini",
    "gemini and laughing about it": "gemini",
    "gemini and somewhat serious about it": "gemini",
    "gemini and very serious about it": "gemini",
    "taurus": "taurus",
    "taurus but not too serious about it": "taurus",
    "taurus and laughing about it": "taurus",
    "taurus and somewhat serious about it": "taurus",
    "taurus and very serious about it": "taurus",
    "pisces": "pisces",
    "pisces but not too serious about it": "pisces",
    "pisces and laughing about it": "pisces",
    "pisces and somewhat serious about it": "pisces",
    "pisces and very serious about it": "pisces",
    "aries": "aries",
    "aries but not too serious about it": "aries",
    "aries and laughing about it": "aries",
    "aries and somewhat serious about it": "aries",
    "aries and very serious about it": "aries",
    "aquarius": "aquarius",
    "aquarius but not too serious about it": "aquarius",
    "aquarius and laughing about it": "aquarius",
    "aquarius and somewhat serious about it": "aquarius",
    "aquarius and very serious about it": "aquarius",
    "sagittarius": "sagittarius",
    "sagittarius but not too serious about it": "sagittarius",
    "sagittarius and laughing about it": "sagittarius",
    "sagittarius and somewhat serious about it": "sagittarius",
    "sagittarius and very serious about it": "sagittarius",
    "capricorn": "capricorn",
    "capricorn but not too serious about it": "capricorn",
    "capricorn and laughing about it": "capricorn",
    "capricorn and somewhat serious about it": "capricorn",
    "capricorn and very serious about it": "capricorn",
}

# Apply mapping directly to the 'sign' column
df['sign'] = df['sign'].str.lower().map(sign_mapping).fillna("unknown")

# Verify results
print(df['sign'].value_counts())

sign
unknown        46873
leo             1142
libra           1084
cancer          1079
virgo           1009
scorpio         1003
gemini           993
taurus           985
pisces           981
aries            981
aquarius         941
sagittarius      924
capricorn        827
Name: count, dtype: int64


In [ ]:
df['speaks'].value_counts()

,count
speaks,
english,21016
english (fluently),6583
"english (fluently), spanish (poorly)",2050
"english (fluently), spanish (okay)",1905
"english (fluently), spanish (fluently)",1275
...,...
"english (fluently), french (okay), italian (okay), hebrew (okay)",1
"english (fluently), farsi (poorly), spanish (poorly), french (poorly)",1
"english (okay), tagalog (okay), japanese (poorly), french (poorly)",1


I want to standardize this column, but I wonder if standardizing will impact PCA results. I will preserve the original column, but create a new column that standardizes the 'speak' column to see if it has an impact on PCA. If not, I will drop the column.

In [ ]:
df['speaks_original'] = df['speaks']

In [ ]:
def categorize_speaks(speaks):
    if pd.isna(speaks):
        return "unknown"  # Handle missing values

    # Remove fluency descriptors (e.g., "(fluently)", "(okay)", "(poorly)")
    cleaned_languages = re.sub(r"\s?\(.*?\)", "", speaks)

    # Convert to lowercase and split into individual languages
    languages = set(cleaned_languages.lower().split(", "))

    # Classify as 'multilingual' if more than one language is listed
    return "monolingual" if len(languages) == 1 else "multilingual"

# Apply function directly to the 'speaks' column
df['speaks'] = df['speaks_original'].apply(categorize_speaks)

In [ ]:
df['speaks'].value_counts()

,count
speaks,
multilingual,30348
monolingual,28433
unknown,41


In [ ]:
df[['speaks_original','speaks']].sample(5)

,speaks_original,speaks
12818,"english (fluently), chinese (fluently), japane...",multilingual
47432,english,monolingual
27883,"english, tagalog (fluently)",multilingual
31519,english,monolingual
12525,"english (fluently), russian (fluently), c++ (f...",multilingual


### Essay Columns

In [ ]:
df['combined_essay_cols'] = df[
    ['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9']
].fillna("").apply(lambda x: " ".join(x), axis=1)

In [ ]:
stop_words = set(stopwords.words("english"))

# Function to clean essay text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

# Apply cleaning to the combined essays
df['cleaned_essays'] = df['combined_essay_cols'].apply(clean_text)

In [ ]:
df.sample(5)

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay5,essay6,essay7,essay8,essay9,income_category,activity_period,speaks_original,combined_essay_cols,cleaned_essays
56681,25,single,m,straight,fit/athletic,anything,socially,never,ph.d_studying,white,...,tasty food good friends and family a challengi...,NaN,eating out or going to a movie with friends,NaN,you find something of interest and want to get...,Unknown,Active in last year,"english (fluently), french (okay), japanese (o...",i'm a well balanced mix of personalities. i'm ...,im well balanced mix personalities im unabashe...
40458,38,single,f,straight,fit/athletic,unknown,often,never,college/university_graduated,indian,...,-my phone -my friends and family -dark chocola...,...keeping things in perspective. even when th...,"...i am where the life takes me, there is alwa...","em...over a four year period, when living in h...",you have new places in the bay area that you c...,Unknown,Active in last year,"english (fluently), french (okay), other (flue...","i am a pretty independant, well travelled, spi...",pretty independant well travelled spiritual in...
42109,19,single,m,straight,fit/athletic,unknown,rarely/not at all,never,college/university_dropped out,white,...,music movies freinds family skateboards stuff ...,pretty much everything that pops into my head....,either working (i work at a comedy club so i'm...,"i'm an open book, i'll tell you anything you w...",you want to get to know me better.,Unknown,Active in last year,english,i'm terrible about talking about myself. i hav...,im terrible talking tattoos make terrible firs...
51506,28,single,f,straight,average,unknown,socially,never,college/university_graduated,white,...,"hot showers, tea, jeans (don't worry, i love d...","the weather, new restaurants, my next workout,...",lately i've been out the door by 8am on saturd...,i will not eat green eggs and ham.,"... you live in san francisco, love travel, ar...",Medium Income,Active in last year,"english (fluently), spanish (fluently)",enjoying summer in new york. catching up with ...,enjoying summer new york catching old friends ...
7530,28,single,f,straight,fit/athletic,vegetarian,socially,unknown,college/university_graduated,unknown,...,"my family/friends, food, music, nature/ocean, ...",how can i help to make the world a better place.,trying to breed a new species of caterpillar.,my second toe is longer than my first.,"you love your mom, your passionate about life,...",Unknown,Active in last year,"english (fluently), german (poorly), spanish (...",im fun. seriously. if your boring or lazy ill ...,im fun seriously boring lazy ill spice life li...


In [ ]:
df.drop(columns=['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9'], inplace=True)

### Final Dataframe

In [ ]:
df.to_csv('okcupid_cleaned.csv', index=False)